In [8]:
from media_impact_monitor.impact import (
    get_impact,
    ImpactSearch,
    correlation,
    time_series_regression,
    interrupted_time_series,
)
from media_impact_monitor.events import get_events, EventSearch
from datetime import date
from media_impact_monitor.trend import get_trend, TrendSearch
import pandas as pd
from great_tables import GT
from IPython.display import display, HTML

organizers = ["Fridays for Future", "Extinction Rebellion", "Greenpeace"]
# organizers = [organizers[0]]
methods = ["correlation", "time_series_regression", "interrupted_time_series"]
# methods = [methods[2]]
start_date = date(2022, 1, 1)
end_date = date(2024, 1, 1)
rows = []
for method in methods:
    trend = get_trend(
        TrendSearch(
            trend_type="keywords",
            media_source="news_print",
            topic="climate_change",
            start_date=start_date,
            end_date=end_date,
        ),
        as_json=False,
    )["climate activism"].rename("count")
    for organizer in organizers:
        events = get_events(
            EventSearch(
                source="acled",
                organizers=[organizer],
                start_date=start_date,
                end_date=end_date,
            )
        )
        aggregation = "daily"
        match method:
            case "correlation":
                row = dict(
                    organizer=organizer,
                    method=method,
                    aggregation=aggregation,
                )
                ts, limitations = correlation(
                    events=events,
                    article_counts=trend,
                )
                row["time_series"] = " ".join(ts["correlation"].values.astype(str))
                rows.append(row)
            case "time_series_regression":
                for lags, ewmas in [
                    ([], []),
                    ([1], []),
                    ([1, 2, 3, 4, 5], []),
                    ([4, 5, 6, 7, 8], []),
                    ([], [1, 7, 28, 84]),
                ]:
                    ts, limitations = time_series_regression(
                        events=events,
                        article_counts=trend,
                        lags=lags,
                    )
                    row = dict(
                        organizer=organizer,
                        method=method,
                        aggregation=aggregation,
                        lags=lags,
                        ewmas=ewmas,
                    )
                    row["rmse"] = ts["rmse"].loc[7]
                    row["rmse_std"] = ts["rmse_std"].loc[7]
                    row["ame"] = ts["ame"].loc[7]
                    row["ame_std"] = ts["ame_std"].loc[7]
                    row["time_series"] = " ".join(ts["mean"].values.astype(str))
                    rows.append(row)
            case "interrupted_time_series":
                for hidden_days_before_protest in [0, 1]:
                    row = dict(
                        organizer=organizer,
                        method=method,
                        aggregation=aggregation,
                        hidden_days_before_protest=hidden_days_before_protest,
                    )
                    ts, lims = interrupted_time_series(
                        events=events,
                        article_counts=trend,
                        horizon=14,
                        hidden_days_before_protest=hidden_days_before_protest,
                        aggregation=aggregation,
                    )
                    row["time_series"] = " ".join(ts["mean"].values.astype(str))
                    rows.append(row)

df = pd.DataFrame(rows)
for organizer in organizers:
    df_ = df[df["organizer"] == organizer].drop(columns="organizer")
    # make "time_series" the last column
    df_ = df_[[c for c in df_.columns if c != "time_series"] + ["time_series"]]
    html = (
        GT(df_)
        .fmt_number(columns=["rmse", "rmse_std", "ame", "ame_std"], decimals=2)
        .fmt_nanoplot(
            columns="time_series",
        )  # restriction in great_tables, only supports one nanoplot column
        .render("html")
    )
    display(HTML(f"<h1>{organizer}</h1>"))
    display(HTML(html))


  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/61 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]